In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom

In [2]:
import os

## Image data is put in folder 'rsna-pneumonia-detection-challenge'

In [3]:
dir_dicom = os.path.join(os.getcwd(), 'rsna-pneumonia-detection-challenge')
os.listdir(dir_dicom)
!ls rsna-pneumonia-detection-challenge

import pandas

GCP Credits Request Link - RSNA.txt stage_2_test_images
stage_2_detailed_class_info.csv     stage_2_train_images
stage_2_sample_submission.csv       stage_2_train_labels.csv


In [4]:
df = pandas.read_csv(os.path.join(dir_dicom, 'stage_2_train_labels.csv'))

In [5]:
df = df.fillna(0)

Patients got diagnosed with numbers of bounding boxs, ranged from 0 to 4

In [6]:
df.groupby("patientId").size().reset_index(name="box_count").groupby("box_count").size()

box_count
1    23286
2     3266
3      119
4       13
dtype: int64

Therefore, do a tranpose and an aggregation

In [7]:
columns = ['x', 'y', 'width', 'height']
df['bounding_boxs'] = df[columns].to_dict(orient='record')
df = df.drop(columns=columns)
df

,patientId,Target,bounding_boxs
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,0,"{'x': 0.0, 'y': 0.0, 'width': 0.0, 'height': 0.0}"
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,0,"{'x': 0.0, 'y': 0.0, 'width': 0.0, 'height': 0.0}"
2,00322d4d-1c29-4943-afc9-b6754be640eb,0,"{'x': 0.0, 'y': 0.0, 'width': 0.0, 'height': 0.0}"
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,0,"{'x': 0.0, 'y': 0.0, 'width': 0.0, 'height': 0.0}"
4,00436515-870c-4b36-a041-de91049b9ab4,1,"{'x': 264.0, 'y': 152.0, 'width': 213.0, 'heig..."
...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,1,"{'x': 185.0, 'y': 298.0, 'width': 228.0, 'heig..."
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,0,"{'x': 0.0, 'y': 0.0, 'width': 0.0, 'height': 0.0}"
30224,c1f6b555-2eb1-4231-98f6-50a963976431,0,"{'x': 0.0, 'y': 0.0, 'width': 0.0, 'height': 0.0}"
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,1,"{'x': 570.0, 'y': 393.0, 'width': 261.0, 'heig..."


In [ ]:
df = df.groupby(["patientId", "Target"])["bounding_boxs"].apply(lambda df: df.reset_index(drop=True)).unstack().reset_index()

In [ ]:
df

In [ ]:
import matplotlib.patches as patches
dicoms_training = os.path.join(dir_dicom, 'stage_2_train_images')

In [ ]:
def print_bounding_box(dicom):
    """plot rectangle boxs for a single dicom image
    params: patientId and bounding boxs       
    """
    _dicom_img = pydicom.dcmread(os.path.join(dicoms_training, dicom.patientId + '.dcm'))
    print("Patient id.......:", dicom.patientId)
    # Create figure and axes
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(_dicom_img.pixel_array, cmap=plt.cm.bone)#cmap stands for color map

    for bb in dicom[-4:]:
        if isinstance(bb, dict) and bb["x"] != "nan":
            x = bb["x"]
            y = bb["y"]
            w = bb["width"]
            h = bb["height"] 
            # Create a Rectangle patch
            rect = patches.Rectangle((x,y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            # Add the patch to the Axes
            ax.add_patch(rect)

    plt.show()

In [ ]:
def print_samples(df, nc = 1):
    count = 0
    for index, row in df.iterrows():
        print_bounding_box(row)
        count +=1
        
        if count >= nc:
            break

## Get some samples of bb(bounding boxs) of 1, 2, 3, 4

In [ ]:
df_4 = df[df[3].notna()]
df_3 = df[df[2].notna()]
df_2 = df[df[1].notna()]
df_1 = df[df[0].notna()]
df_0 = df[df[0] == dict(x=0, y=0, width=0, height=0)]

df_3 = df_3[~df_3.patientId.isin(df_4.patientId)]
df_2 = df_2[~df_2.patientId.isin(df_3.patientId)]
df_1 = df_1[~df_1.patientId.isin(df_2.patientId)]
df_1 = df_1[~df_1.patientId.isin(df_0.patientId)]

In [ ]:
print_samples(df_4)

In [ ]:
print_samples(df_3)

In [ ]:
print_samples(df_2)

In [ ]:
print_samples(df_1)